In [1]:
import pandas as pd 
import requests
import openpyxl
import numpy as np

In [2]:
print('=========> Notation <=========')
print('===> [o] - Success')
print('===> [x] - Error')
print('===> [i] - Ignored')
print('===> [n] - Synset not found')
print('==============================\n')

#Get range for crawler
print("Crwling word range add here (int vals) [end = -1 means till end of data]")
start = int(input("Start of range:"))
end = int(input("End of range:"))

#read fasttext vector
f = open("FastText/cc.en.300.vec",'r', encoding='utf-8')
data = f.readlines()
print('[o] Data loaded Successfully!')

=========> Notation <=========
===> [o] - Success
===> [x] - Error
===> [i] - Ignored
===> [n] - synset not found

Crwling word range add here (int vals) [end = -1 means till end of data]
Start of range:500000
End of range:1000000
[o] Data loaded Successfully!


In [3]:
def html_to_frame(word):     
    url = 'https://www.maduraonline.com/?find='+word
    
    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }
    
    r = requests.get(url, headers=header)

    dfs = pd.read_html(r.text)[0]
    if(len(dfs.columns)>1):
        dfs.drop(columns=[0],inplace=True)
        dfs.dropna(inplace=True)
        dfs[word] =dfs[2].apply(lambda x: x.replace(" ","_")) #dfs[word] = dfs[1]+dfs[2]
        dfs.drop(columns=[1,2],inplace=True)
        return dfs.T
    else:
        return dfs

In [ ]:
total_vector_count = 0
new_vecs = ''

#check user input crawl end
if(end==-1 or end>len(data)):
    end = len(data)

print("\n=====> Crawling Starts <=====\n")
for i in range(start,end):
    try:
        word_vector = data[i].split(" ", 1)
        if(not word_vector[0].isalpha()):
            print("["+str(i)+"][i] | Word: "+word_vector[0])
            continue
        si_words = html_to_frame(word_vector[0])
        si_words_count = len(si_words.columns)
        if(si_words_count>1):
            print("["+str(i)+"][o] | Word: "+word_vector[0])
            temp_vec = ''
            for i in list(dict.fromkeys(si_words.to_string(header=False,index=False).split())):
                total_vector_count +=1 
                temp_vec+= (i+" "+word_vector[1])      
            new_vecs+= temp_vec
        else:
            print("["+str(i)+"][n] | Word: "+word_vector[0])
            continue
    except:
        print("["+str(i)+"][x] | Word: "+str(word_vector[0]))
        continue


=====> Crawling Starts <=====

[500000][n] | Word: Chiarini
[500001][n] | Word: prenomen
[500002][i] | Word: Palestine-related
[500003][n] | Word: NGFW
[500004][n] | Word: Thielmann
[500005][i] | Word: interpretation.
[500006][n] | Word: Olicity
[500007][n] | Word: Bernardston
[500008][i] | Word: Jean-René
[500009][n] | Word: NewsReviewPhonesGiveawayDealsShop
[500010][n] | Word: AgainYesterday
[500011][n] | Word: Dormy
[500012][n] | Word: Bardfield
[500013][i] | Word: km1500
[500014][i] | Word: full-mouth
[500015][n] | Word: Palmwoods
[500016][n] | Word: PONTE
[500017][n] | Word: Glans
[500018][i] | Word: visit.by
[500019][i] | Word: single-layered
[500020][n] | Word: spoilerish
[500021][n] | Word: DealersJunk
[500022][n] | Word: eved
[500023][n] | Word: WELLBreakfastLunchDinnerHealthy
[500024][n] | Word: Meminger
[500025][n] | Word: Heejun
[500026][n] | Word: Videx
[500027][i] | Word: OB1
[500028][n] | Word: goodApr
[500029][i] | Word: -many
[500030][n] | Word: presstitutes
[500031][

In [ ]:
#Write the vec file
to_write = str(total_vector_count)+" "+data[0].split()[1]+'\n'+new_vecs
writer = open("Output/tr.cc.si.300_"+str(start)+"_"+str(end)+".vec", "w", encoding="utf-8")
writer.write(to_write)
writer.close()

#Write new models to combine script
f=open("Output/new_models.txt", "a+")
f.write("tr.cc.si.300_"+str(start)+"_"+str(end)+".vec\n")
f.close()

#final
print("====> Done <====")
print("[o] Total new vectors:",total_vector_count)
print("[o] Vecors saved to 'tr.cc.si.300_"+str(start)+"_"+str(end)+".vec' file successfully.")
